<a href="https://colab.research.google.com/github/B21-CAP0133/verify-android-app/blob/master/ML-dir/08_VERIFY_Build_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

import tensorflow_hub as hub

In [2]:
df = pd.read_csv('verify_dataset_clean.csv')
df.sample(5)

,Judul,Label
6088,tak terima adu soal tarawih remaja rusak rumah...,0
1595,menteri uang sri mulyani milik 1200 bitcoin,1
1175,isis serang kota quaragosh kota populasi krist...,1
531,nissa sabyan bantah undang panitia pesta raya ...,0
5135,judul berita h yakut rencana banser uji nyali ...,1


In [3]:
x_train_text, x_test_text, y_train, y_test = train_test_split(df['Judul'], df['Label'], test_size=0.1)

In [4]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-id-dim128/2",
                           input_shape=[], dtype=tf.string, trainable=True)

In [5]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               112461824 
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 112,461,953
Trainable params: 112,461,953
Non-trainable params: 0
_________________________________________________________________


In [6]:
opt = Adam()
model.compile(optimizer = opt,
              loss = 'binary_crossentropy',
              metrics = ['accuracy']
              )

In [7]:
history = model.fit(x_train_text, y_train,
                    validation_data = (x_test_text, y_test),
                    epochs = 5,
                    shuffle = True
                    )

Epoch 1/5
205/205 [==============================] - 16s 62ms/step - loss: 0.5833 - accuracy: 0.7355 - val_loss: 0.4837 - val_accuracy: 0.7940
Epoch 2/5
205/205 [==============================] - 12s 61ms/step - loss: 0.3654 - accuracy: 0.8650 - val_loss: 0.4281 - val_accuracy: 0.8242
Epoch 3/5
205/205 [==============================] - 13s 61ms/step - loss: 0.2461 - accuracy: 0.9140 - val_loss: 0.4337 - val_accuracy: 0.8201
Epoch 4/5
205/205 [==============================] - 12s 61ms/step - loss: 0.1738 - accuracy: 0.9472 - val_loss: 0.4627 - val_accuracy: 0.8118
Epoch 5/5
205/205 [==============================] - 12s 61ms/step - loss: 0.1249 - accuracy: 0.9659 - val_loss: 0.5083 - val_accuracy: 0.8036


In [8]:
model.save('verify_model_onegig.h5')

In [9]:
export_dir = '/content/'

tf.saved_model.save(model, export_dir)


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: /content/assets


INFO:tensorflow:Assets written to: /content/assets


In [10]:
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [11]:
#model = tf.keras.models.load_model('verify_model_onegig.h5')
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir) # path to the SavedModel directory
converter.optimizations = [optimization]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [13]:
x = 'Jokowi berniat mengundurkan diri dari jabatan presiden bulan depan'
prediksi_hoax = model.predict([x])
prediksi_hoax *= 100
prediksi_hoax

array([[92.22161]], dtype=float32)